# MASK DETECTOR MODEL (LIVE)


In [1]:
# Importing the necessary files :

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import  load_model
from imutils.video import VideoStream

import numpy as np
import imutils
import time
import cv2
import os


In [2]:
def detect_and_predict_mask (frame, facenet, maskNet):
    # grab the dimension of the frame and then construct a blob from it. 
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0,(224,224), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    facenet.setInput(blob)
    detections = facenet.forward ()
    print(detections.shape)

    # initialize our List of faces, their corresponding Locations, and the list of predictions 
    # from our face mask network.

    faces = []
    locs = []
    preds= []

    # loops over the detections :
    for i in range (0, detections.shape[2]):
        # Extract the confidence (i.e. probability ) associated  with the  detection
        confidence = detections[0,0,i,2]

        # filter  out weak detections by ensuring the confidence is greater  than the  minimum confidence 
        if confidence > 0.5:
            # Compute the  (x,y) coordinates  of the bounding box for the object 

            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype('int')

            #  Ensure the bounding boxes fall within the  dimensions of the frame

            (startX,startY) = (max(0,startX), max(0,startY))
            (endX,endY) = (min(w-1,endX), min(h-1,endY))

            # extract the Face ROI, convert it from BGR to RGB channel ordering, resize it to 224X224
            # and process itface =frame[]

            face = frame[startY: endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224,224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the  face and bounding boxes to their respective lists:
            faces.append(face)
            locs.append((startX,startY, endX, endY))

    # only make a prediction if at least one face was detected 
    if len(faces) > 0:
        faces = np.array(faces,dtype="float32")
        preds = maskNet.predict(faces, batch_size = 32)

    
    # return a 2 tuple of the face locations and their corresponding locations  
    return (locs, preds)




In [3]:
# Laod our serialized face detector model from disk
prototxtpath = r'Downlaod\deploy.prototxt'
weights = r'Downlaod\res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(prototxtpath, weights)

# load the face mask detector model from the disk 
maskNet = load_model('mask_detector.model')

In [4]:
# Initializing the video Stream .
print ("[INFO] starting Video Stream...")
vs= VideoStream(src=0).start()

# Initializing the video Stream .
print ("[INFO] starting Video Stream...")
vs= VideoStream(src=0).start()


# Loop over the frames from the video streams: 
while True:
    # Grab the fframe from the threadded video stream and resize it to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width = 800, height = 800)

    # DETECT faces in the frame and determine ifthey are wearing the mask or not:
    (locs , preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # Loop over the detected face Locations and their corresponding Locations 
    for  (box,pred) in zip(locs,preds):
        # unpack the bounding box and predictions 
        (startX, startY, endX, endY ) = box
        (mask, without_mask) = pred
        
        # determine the class label and color we'll use to draw the bounding box and text
        label = 'Mask' if mask> without_mask else "No Mask"
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        # Include the probability in the label: 
        label = "{}: {:.2f}%".format(label, max(mask,without_mask)*100)

        # display the  label and bounding box rectangle on the output frame:
        cv2.putText(frame,label,(startX,startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color,2)
        cv2.rectangle(frame,(startX,startY), (endX,endY),color,2)
    
    # Show the output frarme:
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) %0xFF

    # if the  q has pressed, break from the loop
    if key == ord('q'):
        break

# DOING A BIT OFCLEANUP:
cv2.destroyAllWindows()
vs.stop()

[INFO] starting Video Stream...
[INFO] starting Video Stream...
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1,